# 멜론 주간 Top100 구하기

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import requests 

In [2]:
url = 'https://www.melon.com/chart/week/index.htm'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
req = requests.get(url, headers=headers)
html = req.text

In [3]:
soup = BeautifulSoup(req.text, 'html.parser')

### 시작일 ~ 끝날

In [4]:
soup.select_one('.yyyymmdd').get_text()

'\r\n\t\t\t\t\t\t\t2020.11.09 ~ 2020.11.15\r\n\t\t\t\t\t\t'

In [5]:
date = soup.select_one('.yyyymmdd').get_text().strip()
date

'2020.11.09 ~ 2020.11.15'

In [6]:
start_day = date[:10]
end_day = date[13:]
start_day, end_day

('2020.11.09', '2020.11.15')

### 곡 리스트 찾기

In [7]:
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')
len(trs)

100

In [82]:
rank = int(trs[0].select_one('.rank').string)
rank

1

In [83]:
for i in [0, 1, 2, 19]:
    entry = trs[i].select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
            
    print(diff)

0
1
-1
-100


In [84]:
trs[0].select_one('.rank_wrap')

<span class="rank_wrap" title="순위 동일">
<span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>
<span class="none">0</span>
</span>

In [85]:
span = trs[0].select_one('.rank_wrap')
span.find_all('span')

[<span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>,
 <span class="none">순위 동일</span>,
 <span class="none">0</span>]

In [86]:
span = trs[1].select_one('.rank_wrap')
span.find_all('span')

[<span class="bullet_icons rank_up"><span class="none">단계 상승</span></span>,
 <span class="none">단계 상승</span>,
 <span class="up">1</span>]

In [87]:
span = trs[2].select_one('.rank_wrap')
span.find_all('span')

[<span class="bullet_icons rank_down"><span class="none">단계 하락</span></span>,
 <span class="none">단계 하락</span>,
 <span class="down">1</span>]

In [97]:
info = trs[0].select_one('.ellipsis.rank01')
title = info.find('a').string
info = trs[0].select_one('.ellipsis.rank02')
singer = info.find('a').string
info = trs[0].select_one('.ellipsis.rank03')
album = info.find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

In [99]:
info = trs[0].select('.ellipsis')
title = info[0].find('a').string
singer = info[1].find('a').string
album = info[2].find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

#### DataFrame으로 만들기

In [119]:
ranks = []; diffs = []; titles = []; singers = []; albums = []
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')

In [120]:
for tr in trs:
    rank = int(tr.select_one('.rank').string)

    info = tr.select_one('.ellipsis.rank01')
    title = info.find('a').string
    info = tr.select_one('.ellipsis.rank02')
    singer = info.find('a').string
    info = tr.select_one('.ellipsis.rank03')
    album = info.find('a').string

    entry = tr.select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1


    ranks.append(rank)
    diffs.append(diff)
    titles.append(title)
    singers.append(singer)
    albums.append(album)

In [140]:
top100 = pd.DataFrame({
            'rank': ranks,
            'diff': diffs,
            'title': titles,
            'singer': singers,
            'album': albums 
})
top100 = top100.set_index('rank')
print(f'기간: {start_day} ~ {end_day}')
top100

기간: 2020.11.09 ~ 2020.11.15


,diff,title,singer,album
rank,,,,
1,0,Dynamite,방탄소년단,Dynamite (DayTime Version)
2,1,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다
3,-1,Lovesick Girls,BLACKPINK,THE ALBUM
4,1,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들)
5,-1,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME
...,...,...,...,...
96,-4,화려하지 않은 고백,규현 (KYUHYUN),슬기로운 의사생활 OST Part 4
97,-100,MAGO,여자친구 (GFRIEND),回:Walpurgis Night
98,-3,행복하니,케이시 (Kassy),추(秋)억
